In [2]:
import imgaug.augmenters as iaa
import imgaug.augmentables as iaa_a
import xml.etree.ElementTree as ET
import os
from PIL import Image
import numpy as np

# Define the augmentation sequence
# flipping the horizontally and vertically, rotating, blur
seq = iaa.Sequential([iaa.Flipud(0.5), iaa.Fliplr(0.5), iaa.Affine(rotate=(-10, 10)), iaa.GaussianBlur(sigma=(0, 1.0))])

# Define the input and output directories
input_dir = (r"Dataset\All_Photos_and_XML_Files")
output_dir = (r"Dataset\Augmented_Photo_Files")

# Loop over the input directory
for file_name in os.listdir(input_dir):
    if file_name.endswith('.xml'):
        # Load the image and XML file
        image_file = os.path.splitext(file_name)[0] + '.jpg' #set the image to image_file
        image = Image.open(os.path.join(input_dir, image_file)) #open the image and save in image
        image = np.asarray(image) #change the list of images to a numpy array
        tree = ET.parse(os.path.join(input_dir, file_name))
        root = tree.getroot()

        # Extract the image size from the XML file
        width = int(root.find('size/width').text)
        height = int(root.find('size/height').text)

        # Define the bounding boxes for the XML file
        #all info from XML file parameters
        bboxes = []
        for obj in root.findall('object'):
            xmin = int(obj.find('bndbox/xmin').text)
            ymin = int(obj.find('bndbox/ymin').text)
            xmax = int(obj.find('bndbox/xmax').text)
            ymax = int(obj.find('bndbox/ymax').text)
            bboxes.append(iaa_a.BoundingBox(x1=xmin, y1=ymin, x2=xmax, y2=ymax, label=obj.find('name').text))
            #print(bboxes)
            bboxes = [iaa_a.BoundingBoxesOnImage(bboxes, shape=image.shape)]
            #print(bboxes)
            
        # Apply the augmentation sequence to the image and bounding boxes
        seq_det = seq.to_deterministic()
        image_aug = seq_det.augment_image(image)
        #bboxes_aug = seq_det.augment_bounding_boxes([bboxes], [image])[0]
        bboxes_aug = seq_det.augment_bounding_boxes(bboxes, [image.shape])[0]

        # Update the XML file with the augmented bounding boxes
        for i, obj in enumerate(root.findall('object')):
            obj.find('bndbox/xmin').text = str(int(bboxes_aug.bounding_boxes[i].x1))
            obj.find('bndbox/ymin').text = str(int(bboxes_aug.bounding_boxes[i].y1))
            obj.find('bndbox/xmax').text = str(int(bboxes_aug.bounding_boxes[i].x2))
            obj.find('bndbox/ymax').text = str(int(bboxes_aug.bounding_boxes[i].y2))

#         # Save the augmented image and XML file
#         image_aug = Image.fromarray(image_aug) #convert array back to images
#         image_aug.save(os.path.join(output_dir, image_file)) #save each image
#         tree.write(os.path.join(output_dir, file_name)) #save each XML file
        
        # Save the augmented image and XML file
        for i in range(3):  # create 3 augmented images per input image
            image_aug_filename = os.path.splitext(file_name)[0] + f"_aug{i+1}.jpg"  # create new filename
            image_aug_filename = os.path.join(output_dir, image_aug_filename)
            image_aug_i = seq_det.augment_image(image)  # apply augmentation again
            Image.fromarray(image_aug_i).save(image_aug_filename) #convert array back to images and save
            tree.write(os.path.join(output_dir, file_name)) #save each XML file


KeyboardInterrupt: 